In [417]:
import torch
import os
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import glob 
import cv2
from torch.utils.data import Dataset, DataLoader
from skimage import io
from PIL import Image

# Dataset

In [449]:
tr_path = 'D:\\Finger_Vein_2\\train_img\\' 
ts_path = 'D:\\Finger_Vein_2\\test_img\\' 
train_path=glob.glob(os.path.join(tr_path,"*.jpg"))
test_path = glob.glob(os.path.join(ts_path,"*.jpg"))

class FingerVeinDataset(Dataset):
    #data_path_list - 이미지 path 전체 리스트
    #label - 이미지 ground truth
    def __init__(self, file_path=None, transform=None):
        self.transform = transform
        self.file_path = file_path
        self.img_list = []
        self.label_list= []
        
    # 라벨링  
        for k in range(len(file_path)):
            if 'fake' in file_path[k]:  # fake 1, real 0
                self.label_list.append(1)
                self.img_list.append(file_path[k])
            elif 'real' in file_path[k]:
                self.label_list.append(0)
                self.img_list.append(file_path[k])
#         print(len(self.label_list)) # train: 564 test: 575 잘 나옴

    def __len__(self):
        return len(self.label_list)

    def __getitem__(self, idx):
        label = self.label_list[idx]
        img = Image.open(self.img_list[idx])
        if self.transform:
            img = self.transform(img)  
        return img, label     
        
#         if torch.is_tensor(idx):
#             idx = idx.tolist()
#         image = io.imread(self.path_list[idx])
#         image = torch.from_numpy(image)

#         return image

In [450]:
train_transform = transforms.Compose([transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

In [451]:
train = FingerVeinDataset(transform=train_transform, file_path= train_path)
test = FingerVeinDataset(transform=test_transform, file_path= test_path)

In [452]:
train_data_loader = DataLoader(train,batch_size = 50, shuffle=True, num_workers=0)
test_data_loader = DataLoader(test,batch_size=50, shuffle=False, num_workers=0)

In [453]:
print(len(train_data_loader.dataset)) 
print(len(test_data_loader.dataset)) 

564
575


# VGG16

In [494]:
class VGG16(nn.Module):
    def __init__(self, n_classes):
        super(VGG16, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.a = nn.Conv2d(1, 3, kernel_size=3, padding=1)  # 수정
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.fc6 = nn.Linear(512*1*1, 4096)  # 수정, 25600 
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 2)

    def forward(self, x, training=True):
        x = F.relu(self.a(x))
#         print(x.shape)  # [50, 3, 10, 15]
        x = F.relu(self.conv1_1(x))
#         print(x.shape)  # [50, 64, 10, 15]
        x = F.relu(self.conv1_2(x))
#         print(x.shape)  # [50, 64, 10, 15]
        x = self.pool(x)
#         print(x.shape)  # [50, 64, 5, 7]
        x = F.relu(self.conv2_1(x))
#         print(x.shape)  # [50, 128, 5, 7]
        x = F.relu(self.conv2_2(x))
#         print(x.shape)  # [50, 128, 5, 7]
        x = self.pool(x)
#         print(x.shape)  # [50, 128, 2, 3]
        x = F.relu(self.conv3_1(x))
#         print(x.shape)  # [50, 256, 2, 3]
        x = F.relu(self.conv3_2(x))
#         print(x.shape)  # [50, 256, 2, 3]
        x = F.relu(self.conv3_3(x))
#         print(x.shape)  # [50, 256, 2, 3]
        x = self.pool(x)
#         print(x.shape)  # [50, 256, 1, 1]
        x = F.relu(self.conv4_1(x))
#         print(x.shape)  # [50, 512, 1, 1]
        x = F.relu(self.conv4_2(x))
#         print(x.shape)  # [50, 512, 1, 1]
        x = F.relu(self.conv4_3(x))
#         print(x.shape)  # [50, 512, 1, 1]
#         x = self.pool(x)   # 수정
        x = F.relu(self.conv5_1(x))
#         print(x.shape)  # [50, 512, 1, 1]
        x = F.relu(self.conv5_2(x))
#         print(x.shape)  # [50, 512, 1, 1]
        x = F.relu(self.conv5_3(x))
#         print(x.shape)  # [50, 512, 1, 1]
#         x = self.pool(x)   # 수정
        x = x.view(-1, 512*1*1)  # 수정
#         print(x.shape)  # [50, 512]
        x = F.relu(self.fc6(x))
#         print(x.shape)  # [50, 4096]
        x = F.dropout(x, 0.5, training=training)
#         print(x.shape)  # [50, 4096]
        x = F.relu(self.fc7(x))
#         print(x.shape)  # [50, 4096]
        x = F.dropout(x, 0.5, training=training)
#         print(x.shape)  # [50, 4096]
        x = self.fc8(x)
#         print(x.shape)  # [50, 2]
        return x

In [526]:
vgg16 = VGG16(2)  # 2는 n_classes?
USE_CUDA = torch.cuda.is_available()   # cuda를 쓸 수 있는지 확인하는 코드
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")  # use_cude의 결과에 따라 데이터를 cuda 혹은 cpu로 보내도록 가리키는 역할
# batch_size = 64   
epochs = 40    
learning_rate = 0.0005   

# criterion = nn.BCELoss()
optimizer = optim.Adam(vgg16.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

# Train & Test

In [532]:
# train

def train(vgg16, train_data_loader, optimizer, epoch):
    vgg16.train()
    for batch_idx, (img,target) in enumerate(train_data_loader):
        img, target = img.to(DEVICE), target.to(DEVICE)
#         print(len(target))  # batch_size가 50라서 50 나오는듯. img도 동일하게 50.
        optimizer.zero_grad()
        output = vgg16(img)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 2 == 0 :
            print('train epoch : {}[{}/{} ({: .0f}%)]\tloss:{:.6f}'.format(epoch, len(img), len(train_data_loader.dataset),
                                                                           100* batch_idx / len(train_data_loader.dataset),loss.item()))

In [533]:


def evaluate(vgg16, test_data_loader):
    vgg16.eval()
    test_loss =0   # 테스트 오차 값 0으로 초기화
    correct=0     # 예측이 맞은 수 0으로 초기화
    with torch.no_grad() : # 평가 과정에서는 기울기를 계산하지 않아도 됨
        for img, target in test_data_loader :
            img, target = img.to(DEVICE), target.to(DEVICE)
            output = vgg16(img)
            # 모든 오차 더하기
            test_loss += F.cross_entropy(output,target, reduction = 'sum').item()  #미니배치 평균 대신 합을 받아와야 함
            pred = output.max(1, keepdim = True)[1]  # output.max()함수는 가장 큰 값과 그 값이 있는 인덱스를 출력
            correct +=pred.eq(target.view_as(pred)).sum().item()  #eq()함수는 값이 일치하면 1, 아니면 0을 출력 
    test_loss /= len(test_data_loader.dataset)  # 모델의 전체 데이터셋에 대한 오차를 테스트셋 데이터 수로 나눠 평균 구함
    test_accuracy = 100 * correct / len(test_data_loader.dataset)  # 맞힌 개수의 합을 테스트셋 데이터 수로 나누고 100을 곱해 정확도 구함
    return test_loss, test_accuracy

for epoch in range(1, epochs+1):
    train(vgg16, train_data_loader, optimizer,epoch)
    test_loss, test_accuracy = evaluate(vgg16, test_data_loader)
    print('[{}] test loss : {:.4f}, accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

train epoch : 1[50/564 ( 0%)]	loss:0.691920
train epoch : 1[50/564 ( 0%)]	loss:0.691583
train epoch : 1[50/564 ( 1%)]	loss:0.699683
train epoch : 1[50/564 ( 1%)]	loss:0.693361
train epoch : 1[50/564 ( 1%)]	loss:0.682933
train epoch : 1[50/564 ( 2%)]	loss:0.733695
[1] test loss : 0.6949, accuracy: 51.48%
train epoch : 2[50/564 ( 0%)]	loss:0.698088
train epoch : 2[50/564 ( 0%)]	loss:0.687307
train epoch : 2[50/564 ( 1%)]	loss:0.697053
train epoch : 2[50/564 ( 1%)]	loss:0.698820
train epoch : 2[50/564 ( 1%)]	loss:0.685028
train epoch : 2[50/564 ( 2%)]	loss:0.714726
[2] test loss : 0.6933, accuracy: 52.87%
train epoch : 3[50/564 ( 0%)]	loss:0.701266
train epoch : 3[50/564 ( 0%)]	loss:0.690856
train epoch : 3[50/564 ( 1%)]	loss:0.701291
train epoch : 3[50/564 ( 1%)]	loss:0.694898
train epoch : 3[50/564 ( 1%)]	loss:0.688810
train epoch : 3[50/564 ( 2%)]	loss:0.706827
[3] test loss : 0.6964, accuracy: 48.70%
train epoch : 4[50/564 ( 0%)]	loss:0.685026
train epoch : 4[50/564 ( 0%)]	loss:0.6924

train epoch : 27[50/564 ( 1%)]	loss:0.694706
train epoch : 27[50/564 ( 2%)]	loss:0.696319
[27] test loss : 0.6971, accuracy: 44.52%
train epoch : 28[50/564 ( 0%)]	loss:0.697666
train epoch : 28[50/564 ( 0%)]	loss:0.696707
train epoch : 28[50/564 ( 1%)]	loss:0.697059
train epoch : 28[50/564 ( 1%)]	loss:0.690236
train epoch : 28[50/564 ( 1%)]	loss:0.701303
train epoch : 28[50/564 ( 2%)]	loss:0.691914
[28] test loss : 0.6928, accuracy: 50.26%
train epoch : 29[50/564 ( 0%)]	loss:0.690384
train epoch : 29[50/564 ( 0%)]	loss:0.692623
train epoch : 29[50/564 ( 1%)]	loss:0.696106
train epoch : 29[50/564 ( 1%)]	loss:0.694827
train epoch : 29[50/564 ( 1%)]	loss:0.689239
train epoch : 29[50/564 ( 2%)]	loss:0.691158
[29] test loss : 0.6932, accuracy: 51.30%
train epoch : 30[50/564 ( 0%)]	loss:0.691926
train epoch : 30[50/564 ( 0%)]	loss:0.695792
train epoch : 30[50/564 ( 1%)]	loss:0.697791
train epoch : 30[50/564 ( 1%)]	loss:0.697047
train epoch : 30[50/564 ( 1%)]	loss:0.688202
train epoch : 30[50

**--------------------------------------------------------------------------------**